In [2]:
import pandas as pd
import numpy as np

In [9]:
data_file = "Darsh Suhas Ambade - atom_cord.pdb.txt"

colspecs = [
    (0, 6),   # Record type (ATOM)
    (6, 11),  # Atom serial number
    (12, 16), # Atom name
    (17, 20), # Residue name
    (21, 22), # Chain ID
    (22, 26), # Residue sequence number (RESID)
    (30, 38), # X coordinate
    (38, 46), # Y coordinate
    (46, 54)  # Z coordinate
]

column_names = ["Record", "AtomSerial", "AtomName", "ResName", "ChainID", "ResID", "X", "Y", "Z"]

df = pd.read_fwf("Darsh Suhas Ambade - atom_cord.pdb.txt", colspecs=colspecs, names=column_names)

pdb_dict = {}

for _, row in df.iterrows():
    resid = (row["ResID"])
    atom_name = row["AtomName"].strip()
    res_name = row["ResName"].strip()
    coordinates = [row["X"], row["Y"], row["Z"]]

    if resid not in pdb_dict:
        pdb_dict[resid] = {"resname": res_name}

    pdb_dict[resid][atom_name] = np.array(coordinates)

print(pdb_dict)

{5: {'resname': 'LEU', 'N': array([ 52.074, -24.356,  94.664]), 'CA': array([ 52.675, -23.892,  93.371]), 'C': array([ 51.757, -24.145,  92.178]), 'O': array([ 51.735, -23.358,  91.233]), 'CB': array([ 54.06 , -24.527,  93.144])}, 6: {'resname': 'SER', 'N': array([ 50.975, -25.219,  92.233]), 'CA': array([ 50.019, -25.489,  91.174]), 'C': array([ 49.116, -24.273,  91.082]), 'O': array([ 48.592, -23.981,  90.023]), 'CB': array([ 49.184, -26.734,  91.47 ]), 'OG': array([ 48.14 , -26.895,  90.523])}, 7: {'resname': 'ALA', 'N': array([ 48.953, -23.576,  92.207]), 'CA': array([ 48.171, -22.339,  92.272]), 'C': array([ 48.806, -21.216,  91.454]), 'O': array([ 48.129, -20.576,  90.657]), 'CB': array([ 48.025, -21.898,  93.71 ])}, 8: {'resname': 'GLU', 'N': array([ 50.096, -20.979,  91.71 ]), 'CA': array([ 50.862, -19.946,  91.003]), 'C': array([ 50.913, -20.271,  89.513]), 'O': array([ 50.71 , -19.39 ,  88.664]), 'CB': array([ 52.295, -19.847,  91.564]), 'CG': array([ 53.171, -18.738,  90.936

In [10]:
all_angles = []
for i in pdb_dict:
    N = np.array(pdb_dict[i]["N"])
    CA = np.array(pdb_dict[i]["CA"])
    C = np.array(pdb_dict[i]["C"])

    vec_N_CA = CA - N
    vec_CA_C = C - CA

    angle = np.arccos(np.dot(vec_N_CA, vec_CA_C) / (np.linalg.norm(vec_N_CA) * np.linalg.norm(vec_CA_C)))
    all_angles.append(angle)

print(np.average(all_angles))
print(np.average(np.degrees(all_angles)))

1.2033723406708263
68.94815630321746


In [ ]:

def torsion(p1, p2, p3, p4):
    v1 = p2 - p1
    v2 = p3 - p2
    v3 = p4 - p3

    n1 = np.cross(v1, v2)
    n2 = np.cross(v2, v3)

    norm_n1 = np.linalg.norm(n1)
    norm_n2 = np.linalg.norm(n2)


    n1 = n1/norm_n1 
    n2 = n2/norm_n2

    cos_angle = np.dot(n1, n2)
    angle_rad = np.arccos(cos_angle)

    cross_n1_n2 = np.cross(n1, n2)
    sign = np.dot(cross_n1_n2, v2)

    print(sign)

    angle_rad = angle_rad * sign

    return np.degrees(angle_rad)

phi = []
psi = []
res_ids = sorted(pdb_dict.keys())
for i in range(1, len(res_ids) - 1):

    prev = res_ids[i - 1]
    curr = res_ids[i]
    nxt = res_ids[i + 1]

    if "C" in pdb_dict[prev] and "N" in pdb_dict[curr] and "CA" in pdb_dict[curr] and "C" in pdb_dict[curr]:
        phi.append(torsion(pdb_dict[prev]["C"], pdb_dict[curr]["N"], pdb_dict[curr]["CA"], pdb_dict[curr]["C"]))
        
    if "N" in pdb_dict[curr] and "CA" in pdb_dict[curr] and "C" in pdb_dict[curr] and "N" in pdb_dict[nxt]:
        psi.append(torsion(pdb_dict[curr]["N"], pdb_dict[curr]["CA"], pdb_dict[curr]["C"], pdb_dict[nxt]["N"]))

print("Average Phi Angle:", np.mean(phi))
print("Average Psi Angle:", np.mean(psi))

-1.222557923281424
-0.7151870192555714
-1.3330881608324214
-1.2001543116375026
-1.2948355718177011
-1.041889829418229
-1.4291908632249037
-0.6851242174825116
-1.4131330093544947
-0.8022545033072963
-1.27856130951284
-1.2768317554165998
-1.148384745853967
-1.0900304849585216
-1.3279736740566666
-0.8797270915382855
-1.4026589684532735
-0.8362137485857681
-1.2846805222214306
-1.3210802848412184
-1.0453956472638657
-1.0707514586890337
-1.18986517289897
-1.0512500271469531
-1.3253533668367892
-1.1718616928985433
-1.1143440589461122
-1.320855085973537
-1.1519205135635342
-1.216565332269536
-1.1923380839499877
-0.9027857474128367
-1.4260308922680116
-1.2442377527480692
-0.9786009574134718
-0.9739673868701898
-1.414107332856802
-1.1170842583909741
-1.2903180051481882
-1.02454075901334
-1.24992658674023
-1.3761143336012314
-1.047629152697152
-0.9927600107763749
-1.2935270846187579
-0.9821405083498258
-1.3824925209786834
-1.1832004657793234
-1.0099817959895065
-1.1965422420029714
-1.222460905463